In [1]:
using Pkg
Pkg.activate("../.")

  Activating project at `f:\Programation\ut4m-cardio`


In [2]:
using DataFrames
using XLSX
include("./utils.jl")
include("./preprocessing.jl")
DATA = build_data_cardiac("../Ressources/Données complètes UT4M final120419 avec Data cardio.xlsx")
DATA = DATA[1:end-2,:]
SUB_DATA = DATA[:, [:CODE_SUJET, :COURSE, :TIME, :FC]]

Row,CODE_SUJET,COURSE,TIME,FC
,Int64,Any,Int64,Any
1,1,100,-2,58
2,2,100,-2,60
3,3,100,-2,55
4,4,100,-2,54
5,5,100,-2,50
6,6,100,-2,51
7,7,100,-2,50
8,8,100,-2,66
9,9,100,-2,56


In [3]:
SUB_DATA = dropmissing!(SUB_DATA, :FC)
SUB_DATA = filter!(x -> x.COURSE != "4_40", SUB_DATA)
SUB_DATA.FC = convert(Vector{Float64}, SUB_DATA.FC)
SUB_DATA.TIME = convert(Vector{Float64}, SUB_DATA.TIME)
SUB_DATA

Row,CODE_SUJET,COURSE,TIME,FC
,Int64,Any,Float64,Float64
1,1,100,-2.0,58.0
2,2,100,-2.0,60.0
3,3,100,-2.0,55.0
4,4,100,-2.0,54.0
5,5,100,-2.0,50.0
6,6,100,-2.0,51.0
7,7,100,-2.0,50.0
8,8,100,-2.0,66.0
9,9,100,-2.0,56.0


In [4]:
using MixedModels
using Interpolations
using BasicBSpline

function interpolate(xs::AbstractVector, fs::AbstractVector{T}) where T
    # Cubic open B-spline space
    p = 3
    k = KnotVector(xs) + KnotVector([xs[1],xs[end]]) * p
    P = BSplineSpace{p}(k)

    # dimensions
    m = length(xs)
    n = dim(P)

    # The interpolant function has a f''=0 property at bounds.
    ddP = BSplineDerivativeSpace{2}(P)
    dda = [bsplinebasis(ddP,j,xs[1]) for j in 1:n]
    ddb = [bsplinebasis(ddP,j,xs[m]) for j in 1:n]

    # Compute the interpolant function (1-dim B-spline manifold)
    M = [bsplinebasis(P,j,xs[i]) for i in 1:m, j in 1:n]
    M = vcat(dda', M, ddb')
    y = vcat(zero(T), fs, zero(T))
    return BSplineManifold(M\y, P)
end



interpolate (generic function with 1 method)

In [6]:
using Statistics

mean_list = combine(groupby(SUB_DATA, :TIME), :FC => mean => :mean_FC)
x = mean_list.TIME
y = mean_list.mean_FC
f = interpolate(x,y)

BSplineManifold{1, (3,), Float64, Float64, Tuple{BSplineSpace{3, Float64, KnotVector{Float64}}}}([57.85964912280702, 71.66181745608264, 94.6654313448753, 44.81027017952703, 62.147178987165425, 61.32400657015521, 60.80952380952381], (BSplineSpace{3, Float64, KnotVector{Float64}}(KnotVector([-2.0, -2.0, -2.0, -2.0, 1.0, 3.0, 6.0, 11.0, 11.0, 11.0, 11.0])),))

In [7]:
model = fit(
    MixedModel,
    @formula(FC ~ -1 + COURSE + f(TIME) + (1 | CODE_SUJET)),
    SUB_DATA
)

|             |    Est. |     SE |     z |      p | σ_CODE_SUJET |
|:----------- | -------:| ------:| -----:| ------:| ------------:|
| COURSE: 40  |  0.9262 | 3.8882 |  0.24 | 0.8117 |              |
| COURSE: 100 | -3.1430 | 4.0862 | -0.77 | 0.4418 |              |
| COURSE: 160 |  0.1694 | 3.6551 |  0.05 | 0.9630 |              |
| f(TIME)     |  1.0036 | 0.0539 | 18.62 | <1e-76 |              |
| (Intercept) |         |        |       |        |       7.1857 |
| Residual    |  6.9051 |        |       |        |              |


In [9]:
println(model)

Linear mixed model fit by maximum likelihood
 FC ~ 0 + COURSE + :(f(TIME)) + (1 | CODE_SUJET)
   logLik   -2 logLik     AIC       AICc        BIC    
  -805.9977  1611.9955  1623.9955  1624.3773  1644.5452

Variance components:
              Column   Variance Std.Dev.
CODE_SUJET (Intercept)  51.63395 7.18568
Residual                47.68075 6.90513
 Number of obs: 227; levels of grouping factors: 57

  Fixed-effects parameters:
───────────────────────────────────────────────────
                 Coef.  Std. Error      z  Pr(>|z|)
───────────────────────────────────────────────────
COURSE: 40    0.926241   3.88816     0.24    0.8117
COURSE: 100  -3.143      4.0862     -0.77    0.4418
COURSE: 160   0.16944    3.65506     0.05    0.9630
f(TIME)       1.0036     0.0538941  18.62    <1e-76
───────────────────────────────────────────────────
